In [ ]:
import pyodbc
import gradio as gr
import pandas as pd
from dotenv import load_dotenv
import os

from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain_openai import OpenAI
# from langchain.agents import AgentExecutor

# Load environment variables from .env file
load_dotenv()
# Access variables
server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('PASSWORD')
openai_api_key=os.getenv("OPENAI_API_KEY")

connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

db = SQLDatabase.from_uri(connection_string)

# Create the SQLDatabaseToolkit using the initialized SQLDatabase object and an OpenAI instance
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0,openai_api_key=openai_api_key))

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0, openai_api_key=openai_api_key),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # You can choose the appropriate agent type here
)

def sql_chat(question):
    return agent_executor.run(question)

#Print out the original data using the language in SQL
def fetch_data():
    connection_string_fetch_data = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    cnxn = pyodbc.connect(connection_string_fetch_data)
    cursor = cnxn.cursor()

    query = "SELECT * FROM ERPobjects"
    
    cursor.execute(query)

    # Fetch data and column names
    data = cursor.fetchall()

    # Convert to pandas DataFrame
    df = pd.DataFrame.from_records(data, columns=[column[0] for column in cursor.description])

    # Close cursor and connection
    cursor.close()
    cnxn.close()

    return df

# Create a Gradio interface with Blocks
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            # Button for fetching data
            fetch_button = gr.Button("Fetch Origin Data")
            # Output textbox for fetch data
            fetch_output = gr.Dataframe()
        with gr.Column():
            with gr.Row():
                # Textbox for entering SQL query
                query_input = gr.Textbox(label="Enter your SQL query")
                # Button for SQL chat
                chat_button = gr.Button("Ask SQL Agent")
            # Output textbox for SQL chat
            chat_output = gr.Textbox(label="SQL Agent Response")
        
    # Define actions
    fetch_button.click(fn=fetch_data, outputs=fetch_output)
    chat_button.click(fn=sql_chat, inputs=query_input, outputs=chat_output)


# demo.launch(share=True, auth=("username", "password"))
demo.launch()

In [ ]:
print(db.table_info)

Adding Prompt

In [ ]:
import pyodbc
import gradio as gr
import pandas as pd
from dotenv import load_dotenv
import os

from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain_openai import OpenAI

# Load environment variables from .env file
load_dotenv()
# Access variables
server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('PASSWORD')
openai_api_key=os.getenv("OPENAI_API_KEY")
print(openai_api_key)
connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

db = SQLDatabase.from_uri(connection_string)

# Create the SQLDatabaseToolkit using the initialized SQLDatabase object and an OpenAI instance
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0,openai_api_key=openai_api_key))

from langchain.prompts import PromptTemplate

TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the reply based on the results of the query.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Custom Database Description:
{custom_database_description}

Only use the following tables:

{table_info}.

Question: {input}"""

# Define your custom database description
custom_database_description = """
Table: erp
Columns: id (int), name (varchar), stock (int)

"""


# Replace the original CUSTOM_PROMPT with the updated prompt
CUSTOM_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect", "custom_database_description"],
    template=TEMPLATE,
)

# Pass the custom prompt to your LangChain setup
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0, openai_api_key=openai_api_key),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    prompt_template=CUSTOM_PROMPT,
    custom_database_description=custom_database_description,
)

def sql_chat(question):
    return agent_executor.run(question)

#Print out the original data using the language in SQL
def fetch_data():
    connection_string_fetch_data = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    cnxn = pyodbc.connect(connection_string_fetch_data)
    cursor = cnxn.cursor()

    query = "SELECT * FROM ERPobjects"
    cursor.execute(query)

    # Fetch data and column names
    data = cursor.fetchall()

    # Convert to pandas DataFrame
    df = pd.DataFrame.from_records(data, columns=[column[0] for column in cursor.description])

    # Close cursor and connection
    cursor.close()
    cnxn.close()

    return df


# use gradio to create an interface for sql agent
# demo = gr.Interface(
#     fn=sql_chat, 
#     inputs="textbox",
#     outputs="textbox"
# )
# Create a Gradio interface with Blocks
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            # Button for fetching data
            fetch_button = gr.Button("Fetch Origin Data")
            # Output textbox for fetch data
            fetch_output = gr.Dataframe()
        with gr.Column():
            with gr.Row():
                # Textbox for entering SQL query
                query_input = gr.Textbox(label="Enter your SQL query")
                # Button for SQL chat
                chat_button = gr.Button("Ask SQL Agent")
            # Output textbox for SQL chat
            chat_output = gr.Textbox(label="SQL Agent Response")
        
    # Define actions
    fetch_button.click(fn=fetch_data, outputs=fetch_output)
    chat_button.click(fn=sql_chat, inputs=query_input, outputs=chat_output)


# demo.launch(share=True, auth=("username", "password"))
demo.launch()

Getting all the tables

In [26]:
import pyodbc
import gradio as gr
import pandas as pd
from dotenv import load_dotenv
import os

from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent, AgentType
from langchain_openai import OpenAI
# from langchain.agents import AgentExecutor

# Load environment variables from .env file
load_dotenv()
# Access variables
server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('PASSWORD')
openai_api_key=os.getenv("OPENAI_API_KEY")

connection_string = f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server"

db = SQLDatabase.from_uri(connection_string)

# Create the SQLDatabaseToolkit using the initialized SQLDatabase object and an OpenAI instance
# toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0,openai_api_key=openai_api_key))

# agent_executor = create_sql_agent(
#     llm=OpenAI(temperature=0, openai_api_key=openai_api_key),
#     toolkit=toolkit,
#     verbose=True,
#     agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # You can choose the appropriate agent type here
# )

# def sql_chat(question):
#     return agent_executor.run(question)

#Print out the original data using the language in SQL
def fetch_data():
    connection_string_fetch_data = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
    cnxn = pyodbc.connect(connection_string_fetch_data)
    cursor = cnxn.cursor()

    # Query to get all table names
    table_query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"

    # Execute the table query
    cursor.execute(table_query)

    # Fetch table names
    table_names = cursor.fetchall()

    # Initialize an empty dictionary to store DataFrames
    df_dict = {}

    # Iterate through each table and fetch data
    for table_name in table_names:
        # Build the query for each table
        query = f"SELECT * FROM {table_name[0]}"
        
        # Execute the query
        cursor.execute(query)
        
        # Fetch data
        data = cursor.fetchall()
        
        # Convert to pandas DataFrame
        df = pd.DataFrame.from_records(data, columns=[column[0] for column in cursor.description])
        
        # Store the DataFrame in the dictionary with the table name as the key
        df_dict[table_name[0]] = df

    # Close cursor and connection
    cursor.close()
    cnxn.close()

    return df_dict

result_dict = fetch_data()

# Access individual DataFrames using table names
for table_name, df in result_dict.items():
    print(f"Table: {table_name}")
    print(df)
    print("\n")


# Create a Gradio interface with Blocks
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            # Button for fetching data
            fetch_button = gr.Button("Fetch Origin Data")
            # Output textbox for fetch data
            fetch_output = gr.Dataframe()
        # with gr.Column():
        #     with gr.Row():
        #         # Textbox for entering SQL query
        #         query_input = gr.Textbox(label="Enter your SQL query")
        #         # Button for SQL chat
        #         chat_button = gr.Button("Ask SQL Agent")
        #     # Output textbox for SQL chat
        #     chat_output = gr.Textbox(label="SQL Agent Response")
        
    # Define actions
    fetch_button.click(fn=fetch_data, outputs=fetch_output)
    chat_button.click(fn=sql_chat, inputs=query_input, outputs=chat_output)


# demo.launch(share=True, auth=("username", "password"))
demo.launch()

    id   name  stock username gender principal_id diagram_id version  \
0  1.0  P0001   10.0      NaN    NaN          NaN        NaN     NaN   
1  2.0  P0002   15.0      NaN    NaN          NaN        NaN     NaN   
2  3.0  P0003    5.0      NaN    NaN          NaN        NaN     NaN   
3  4.0  S0001  100.0      NaN    NaN          NaN        NaN     NaN   
4  5.0  S0002  150.0      NaN    NaN          NaN        NaN     NaN   
5  6.0  S0003   50.0      NaN    NaN          NaN        NaN     NaN   
6  7.0  T0001  123.0      NaN    NaN          NaN        NaN     NaN   
7  1.0    NaN    NaN     John      m          NaN        NaN     NaN   
8  2.0    NaN    NaN      Tom      m          NaN        NaN     NaN   
9  3.0    NaN    NaN      Sue      f          NaN        NaN     NaN   

  definition  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
5        NaN  
6        NaN  
7        NaN  
8        NaN  
9        NaN  
Running on local URL:  http://127.0.0.1:78